In [1]:
!pip install scrapy

  Using cached Scrapy-2.6.1-py2.py3-none-any.whl (264 kB)
  Using cached lxml-4.8.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.9 MB)
  Using cached itemadapter-0.6.0-py3-none-any.whl (10 kB)
  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached Twisted-22.4.0-py3-none-any.whl (3.1 MB)
  Using cached tldextract-3.3.0-py3-none-any.whl (93 kB)
  Using cached itemloaders-1.0.4-py3-none-any.whl (11 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached PyDispatcher-2.0.5-py3-none-any.whl
  Using cached service_identity-21.1.0-py2.py3-none-any.whl (12 kB)
  Using cached queuelib-1.6.2-py2.py3-none-any.whl (13 kB)
  Using cached zope.interface-5.4.0-cp39-cp39-manylinux2010_x86_64.whl (255 kB)
  Using cached Protego-0.2.1-py2.py3-none-any.whl (8.2 kB)
  Using cached parsel-1.6.0-py2.py3-none-any.whl (13 kB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 

In [1]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"] #la liste des villes dont nous allons chercher les informations

cities_without_spaces = [x.replace(" ","+") for x in cities]
cities_without_spaces #les espaces sont remplacés par des +, cela nous sera utile pour la suite
cities_for_dic = [x.replace(" ","_") for x in cities]

r = requests.get("https://nominatim.openstreetmap.org/search.php?q=Mont+Saint+Michel&format=jsonv2")
#premier test pour récupérer les infos pour le mont saint michel
infos_test = r.json()
#infos_test #les infos dont on a besoin sont les premières

urls_to_get = []

for x in range(len(cities_without_spaces)):
    urls_to_get.append("https://nominatim.openstreetmap.org/search.php?q={}&format=jsonv2".format(cities_without_spaces[x]))
    

dataset = pd.DataFrame(columns = infos_test[0].keys())
#dataset.head() #Nous avons maintenant la base pour note Dataset

liste_infos = []

for x in range(len(urls_to_get)):
    r = requests.get(urls_to_get[x])
    liste_infos.append(r.json()[0])
    
dataset = dataset.append(liste_infos)
#dataset.head() #nous n'allons garder que le display_name, la latitude et la longitue


columns_to_keep = ['lat', 'lon', 'display_name']
dataset = dataset.drop((c for c in dataset.columns if c not in columns_to_keep), axis=1)
dataset['id'] = dataset.index
dataset['cities'] = cities
dataset = dataset[['id','cities','lat','lon']]
dataset.head()

/tmp/ipykernel_196/1302081398.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(liste_infos)


,id,cities,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


In [3]:
# Nous allons maintenant récupérer des informations sur les météos des 35 villes 
#url_type_for_weather = "https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}"

my_key_for_API_weather = "9dbefcbf7b6ac43109f987d714de8e50"

weathers = []

for x in dataset.index:
    r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={dataset.lat[x]}&lon={dataset.lon[x]}&exclude=current,minutely,hourly,alerts&appid={my_key_for_API_weather}")
    weathers.append(r.json())

In [4]:
print(type(weathers))
print()
print(len(weathers))
print()
print(type(weathers[0]['daily']))
print()
print(weathers[0]['daily'][0]['weather'][0]['main'])

<class 'list'>

35

<class 'list'>

Rain


In [5]:
weathers_week = [x for x in range(0,35)] #je crée une liste de 35 éléments
for x in weathers_week:
    weathers_week[x] = [] #je transforme chaque élément en liste vide

for x in range(len(weathers_week)):
    for y in weathers[x]['daily']:
        weathers_week[x].append(y['weather'][0]['main'])

In [32]:
#  C = K - 273.15. va nous permettre de convertir les températures en celsius
weathers_temp = [x for x in range(0,35)] #je crée une liste de 35 éléments
for x in weathers_temp:
    weathers_temp[x] = [] #je transforme chaque élément en liste vide
    
for x in range(len(weathers_temp)):
    for y in weathers[x]['daily']:
        weathers_temp[x].append(y['temp']['day']-273.15)

In [33]:
weathers_temp

[[19.07000000000005,
  24.32000000000005,
  21.5,
  17.02000000000004,
  18.260000000000048,
  19.620000000000005,
  23.689999999999998,
  16.460000000000036],
 [19.689999999999998,
  21.82000000000005,
  18.80000000000001,
  15.950000000000045,
  17.25,
  17.99000000000001,
  20.28000000000003,
  15.610000000000014],
 [19.860000000000014,
  22.480000000000018,
  20.370000000000005,
  17.28000000000003,
  16.450000000000045,
  19.600000000000023,
  19.210000000000036,
  17.100000000000023],
 [18.25,
  20.370000000000005,
  17.5,
  15.050000000000011,
  15.120000000000005,
  16.180000000000007,
  18.140000000000043,
  14.450000000000045],
 [21.400000000000034,
  22.890000000000043,
  22.360000000000014,
  18.850000000000023,
  16.950000000000045,
  19.920000000000016,
  19.730000000000018,
  17.54000000000002],
 [23.110000000000014,
  25.210000000000036,
  25.930000000000007,
  24.260000000000048,
  21.680000000000007,
  22.980000000000018,
  24.30000000000001,
  20.590000000000032],
 [

In [34]:
print(weathers_week[0])
print()
print(weathers_week[1])
print()
print(weathers_week[2])
print()
print(weathers_week[3])

['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Clouds', 'Rain', 'Clouds']

['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Clouds', 'Rain', 'Clouds']

['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Clouds', 'Rain', 'Clouds']

['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Clouds', 'Rain', 'Clouds']


In [35]:
weathers_week[20]

['Clouds', 'Clear', 'Clouds', 'Clouds', 'Clouds', 'Clouds', 'Clouds', 'Clouds']

In [36]:
cities_for_dic = [x.replace(" ","_") for x in cities]
weather_dic = dict(zip(cities_for_dic, weathers_week))
#dictionnaire avec les villes en clés et la météo de la semaine comme values

In [37]:
weather_dic['Mont_Saint_Michel']

['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Clouds', 'Rain', 'Clouds']

In [38]:
dataset.head()

,id,cities,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


In [45]:
days = [x for x in range(1,9)]
for x in range(len(days)):
    days[x] = "day " + str(x+1)
print(days)

temp = [x for x in range(1,9)]
for x in range(len(temp)):
    temp[x] = "day " + str(x+1) + " °C"
print(temp)

['day 1', 'day 2', 'day 3', 'day 4', 'day 5', 'day 6', 'day 7', 'day 8']
['day 1 °C', 'day 2 °C', 'day 3 °C', 'day 4 °C', 'day 5 °C', 'day 6 °C', 'day 7 °C', 'day 8 °C']


In [48]:
weather_df = pd.DataFrame(index=weather_dic.keys(), data=weathers_week)
weather_df.columns = days
display(weather_df.head())
print()
temp_df = pd.DataFrame(index=weather_dic.keys(), data=weathers_temp)
temp_df.columns = temp
display(temp_df.head())

,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8
Mont_Saint_Michel,Rain,Rain,Rain,Rain,Rain,Clouds,Rain,Clouds
St_Malo,Rain,Rain,Rain,Rain,Rain,Clouds,Rain,Clouds
Bayeux,Rain,Rain,Rain,Rain,Rain,Clouds,Rain,Clouds
Le_Havre,Rain,Rain,Rain,Rain,Rain,Clouds,Rain,Clouds
Rouen,Rain,Rain,Rain,Rain,Rain,Clouds,Rain,Rain


,day 1 °C,day 2 °C,day 3 °C,day 4 °C,day 5 °C,day 6 °C,day 7 °C,day 8 °C
Mont_Saint_Michel,19.07,24.32,21.50,17.02,18.26,19.62,23.69,16.46
St_Malo,19.69,21.82,18.80,15.95,17.25,17.99,20.28,15.61
Bayeux,19.86,22.48,20.37,17.28,16.45,19.60,19.21,17.10
Le_Havre,18.25,20.37,17.50,15.05,15.12,16.18,18.14,14.45
Rouen,21.40,22.89,22.36,18.85,16.95,19.92,19.73,17.54
